# Analisi dell'impatto ambientale della produzione di cibi e mangimi
### In base ai dati raccolti dall' Organizzazione delle Nazioni Unite capiremo quali sono gli alimenti che richiedono più risorse naturali in modo tale da poter spostare il focus produttivo su alimenti più sostenibili per l'ambiente.

In [58]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re as regex


FAO_df = pd.read_csv("data/FAO.csv", encoding='latin-1')
Food_Production_df = pd.read_csv("data/Food_Production.csv")
FAO_df["Type"] = ""

## Visualizzazione dei tipi di dato a disposizione

In [59]:
FAO_df.columns

Index(['Area Abbreviation', 'Area Code', 'Area', 'Item Code', 'Item',
       'Element Code', 'Element', 'Unit', 'latitude', 'longitude', 'Y1961',
       'Y1962', 'Y1963', 'Y1964', 'Y1965', 'Y1966', 'Y1967', 'Y1968', 'Y1969',
       'Y1970', 'Y1971', 'Y1972', 'Y1973', 'Y1974', 'Y1975', 'Y1976', 'Y1977',
       'Y1978', 'Y1979', 'Y1980', 'Y1981', 'Y1982', 'Y1983', 'Y1984', 'Y1985',
       'Y1986', 'Y1987', 'Y1988', 'Y1989', 'Y1990', 'Y1991', 'Y1992', 'Y1993',
       'Y1994', 'Y1995', 'Y1996', 'Y1997', 'Y1998', 'Y1999', 'Y2000', 'Y2001',
       'Y2002', 'Y2003', 'Y2004', 'Y2005', 'Y2006', 'Y2007', 'Y2008', 'Y2009',
       'Y2010', 'Y2011', 'Y2012', 'Y2013', 'Type'],
      dtype='object')

In [60]:
Food_Production_df.columns

Index(['Food product', 'Land use change', 'Animal Feed', 'Farm', 'Processing',
       'Transport', 'Packging', 'Retail', 'Total_emissions',
       'Eutrophying emissions per 1000kcal (gPO₄eq per 1000kcal)',
       'Eutrophying emissions per kilogram (gPO₄eq per kilogram)',
       'Eutrophying emissions per 100g protein (gPO₄eq per 100 grams protein)',
       'Freshwater withdrawals per 1000kcal (liters per 1000kcal)',
       'Freshwater withdrawals per 100g protein (liters per 100g protein)',
       'Freshwater withdrawals per kilogram (liters per kilogram)',
       'Greenhouse gas emissions per 1000kcal (kgCO₂eq per 1000kcal)',
       'Greenhouse gas emissions per 100g protein (kgCO₂eq per 100g protein)',
       'Land use per 1000kcal (m² per 1000kcal)',
       'Land use per kilogram (m² per kilogram)',
       'Land use per 100g protein (m² per 100g protein)',
       'Scarcity-weighted water use per kilogram (liters per kilogram)',
       'Scarcity-weighted water use per 100g protein 

## Campioni di valori presenti nei datasets

In [61]:
FAO_df

,Area Abbreviation,Area Code,Area,Item Code,Item,Element Code,Element,Unit,latitude,longitude,...,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013,Type
0,AFG,2,Afghanistan,2511,Wheat and products,5142,Food,1000 tonnes,33.94,67.71,...,3486.0,3704.0,4164.0,4252.0,4538.0,4605.0,4711.0,4810,4895,
1,AFG,2,Afghanistan,2805,Rice (Milled Equivalent),5142,Food,1000 tonnes,33.94,67.71,...,445.0,546.0,455.0,490.0,415.0,442.0,476.0,425,422,
2,AFG,2,Afghanistan,2513,Barley and products,5521,Feed,1000 tonnes,33.94,67.71,...,236.0,262.0,263.0,230.0,379.0,315.0,203.0,367,360,
3,AFG,2,Afghanistan,2513,Barley and products,5142,Food,1000 tonnes,33.94,67.71,...,43.0,44.0,48.0,62.0,55.0,60.0,72.0,78,89,
4,AFG,2,Afghanistan,2514,Maize and products,5521,Feed,1000 tonnes,33.94,67.71,...,208.0,233.0,249.0,247.0,195.0,178.0,191.0,200,200,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21472,ZWE,181,Zimbabwe,2948,Milk - Excluding Butter,5142,Food,1000 tonnes,-19.02,29.15,...,357.0,359.0,356.0,341.0,385.0,418.0,457.0,426,451,
21473,ZWE,181,Zimbabwe,2960,"Fish, Seafood",5521,Feed,1000 tonnes,-19.02,29.15,...,4.0,9.0,6.0,9.0,5.0,15.0,15.0,15,15,
21474,ZWE,181,Zimbabwe,2960,"Fish, Seafood",5142,Food,1000 tonnes,-19.02,29.15,...,14.0,17.0,14.0,15.0,18.0,29.0,40.0,40,40,
21475,ZWE,181,Zimbabwe,2961,"Aquatic Products, Other",5142,Food,1000 tonnes,-19.02,29.15,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,


In [62]:
Food_Production_df

,Food product,Land use change,Animal Feed,Farm,Processing,Transport,Packging,Retail,Total_emissions,Eutrophying emissions per 1000kcal (gPO₄eq per 1000kcal),...,Freshwater withdrawals per 100g protein (liters per 100g protein),Freshwater withdrawals per kilogram (liters per kilogram),Greenhouse gas emissions per 1000kcal (kgCO₂eq per 1000kcal),Greenhouse gas emissions per 100g protein (kgCO₂eq per 100g protein),Land use per 1000kcal (m² per 1000kcal),Land use per kilogram (m² per kilogram),Land use per 100g protein (m² per 100g protein),Scarcity-weighted water use per kilogram (liters per kilogram),Scarcity-weighted water use per 100g protein (liters per 100g protein),Scarcity-weighted water use per 1000kcal (liters per 1000 kilocalories)
0,Wheat & Rye (Bread),0.1,0.0,0.8,0.2,0.1,0.1,0.1,1.4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Maize (Meal),0.3,0.0,0.5,0.1,0.1,0.1,0.0,1.1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Barley (Beer),0.0,0.0,0.2,0.1,0.0,0.5,0.3,1.1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Oatmeal,0.0,0.0,1.4,0.0,0.1,0.1,0.0,1.6,4.281357,...,371.076923,482.4,0.945482,1.907692,2.897446,7.60,5.846154,18786.2,14450.923080,7162.104461
4,Rice,0.0,0.0,3.6,0.1,0.1,0.1,0.1,4.0,9.514379,...,3166.760563,2248.4,1.207271,6.267606,0.759631,2.80,3.943662,49576.3,69825.774650,13449.891480
5,Potatoes,0.0,0.0,0.2,0.0,0.1,0.0,0.0,0.3,4.754098,...,347.647059,59.1,0.628415,2.705882,1.202186,0.88,5.176471,2754.2,16201.176470,3762.568306
6,Cassava,0.6,0.0,0.2,0.0,0.1,0.0,0.0,0.9,0.708419,...,NaN,0.0,1.355236,14.666667,1.858316,1.81,20.111111,0.0,NaN,NaN
7,Cane Sugar,1.2,0.0,0.5,0.0,0.8,0.1,0.0,2.6,4.820513,...,NaN,620.1,0.911681,NaN,0.581197,2.04,NaN,16438.6,NaN,4683.361823
8,Beet Sugar,0.0,0.0,0.5,0.2,0.6,0.1,0.0,1.4,1.541311,...,NaN,217.7,0.515670,NaN,0.521368,1.83,NaN,9493.3,NaN,2704.643875
9,Other Pulses,0.0,0.0,1.1,0.0,0.1,0.4,0.0,1.6,5.008798,...,203.503036,435.7,0.524927,0.836058,4.565982,15.57,7.272303,22477.4,10498.552080,NaN


# Fenomeni e colonne

### Uno dei fenomeni da tenere sicuramente in considerazione è l'eutrofizzazione (Eutrophying) che porta alla morte di flora e fauna marina. Quindi sarà un dato che prenderemo in considerazione.
### Le colonne dove non è segnata l'unità di misura a livello di consumo ci si riferisce al consumo in KG di CO2

# Colonne del primo dataset
### Del primo dataset andremo a considerare le seguenti colonne:
#### - Area Abbreviation -> Abbreviazione dell'area
#### - Area -> Area scritta per esteso
#### - Item -> Prodotto
#### - Element -> Mangime o Cibo
#### - Unit -> Unità di misura per la produzione
#### - Y* (Tutte le colonne che iniziano con Y) -> Quanto è stato prodotto ogni anno

In [63]:
# Selezione delle colonne del primo dataset
colonne_interessate = ["Area Abbreviation", "Area", "Item", "Element", "Unit", "Type"]
FAO_df_colonne_interessate = [ colonna for colonna in FAO_df.columns if colonna.startswith("Y") or (colonna in colonne_interessate)]
FAO_df[FAO_df_colonne_interessate]

,Area Abbreviation,Area,Item,Element,Unit,Y1961,Y1962,Y1963,Y1964,Y1965,...,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013,Type
0,AFG,Afghanistan,Wheat and products,Food,1000 tonnes,1928.0,1904.0,1666.0,1950.0,2001.0,...,3486.0,3704.0,4164.0,4252.0,4538.0,4605.0,4711.0,4810,4895,
1,AFG,Afghanistan,Rice (Milled Equivalent),Food,1000 tonnes,183.0,183.0,182.0,220.0,220.0,...,445.0,546.0,455.0,490.0,415.0,442.0,476.0,425,422,
2,AFG,Afghanistan,Barley and products,Feed,1000 tonnes,76.0,76.0,76.0,76.0,76.0,...,236.0,262.0,263.0,230.0,379.0,315.0,203.0,367,360,
3,AFG,Afghanistan,Barley and products,Food,1000 tonnes,237.0,237.0,237.0,238.0,238.0,...,43.0,44.0,48.0,62.0,55.0,60.0,72.0,78,89,
4,AFG,Afghanistan,Maize and products,Feed,1000 tonnes,210.0,210.0,214.0,216.0,216.0,...,208.0,233.0,249.0,247.0,195.0,178.0,191.0,200,200,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21472,ZWE,Zimbabwe,Milk - Excluding Butter,Food,1000 tonnes,230.0,232.0,234.0,238.0,234.0,...,357.0,359.0,356.0,341.0,385.0,418.0,457.0,426,451,
21473,ZWE,Zimbabwe,"Fish, Seafood",Feed,1000 tonnes,27.0,25.0,27.0,23.0,27.0,...,4.0,9.0,6.0,9.0,5.0,15.0,15.0,15,15,
21474,ZWE,Zimbabwe,"Fish, Seafood",Food,1000 tonnes,6.0,6.0,6.0,8.0,9.0,...,14.0,17.0,14.0,15.0,18.0,29.0,40.0,40,40,
21475,ZWE,Zimbabwe,"Aquatic Products, Other",Food,1000 tonnes,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,


### Del secondo dataset prendiamo tutto e non avremo bisogno di effetuare operazioni di filtraggio

### Estrapolazione degli alimenti il cui inquinamento è tracciato


In [64]:
cibi = Food_Production_df["Food product"]
cibi

0     Wheat & Rye (Bread)
1            Maize (Meal)
2           Barley (Beer)
3                 Oatmeal
4                    Rice
5                Potatoes
6                 Cassava
7              Cane Sugar
8              Beet Sugar
9            Other Pulses
10                   Peas
11                   Nuts
12             Groundnuts
13                Soymilk
14                   Tofu
15            Soybean Oil
16               Palm Oil
17          Sunflower Oil
18           Rapeseed Oil
19              Olive Oil
20               Tomatoes
21         Onions & Leeks
22        Root Vegetables
23              Brassicas
24       Other Vegetables
25           Citrus Fruit
26                Bananas
27                 Apples
28       Berries & Grapes
29                   Wine
30            Other Fruit
31                 Coffee
32         Dark Chocolate
33       Beef (beef herd)
34      Beef (dairy herd)
35          Lamb & Mutton
36               Pig Meat
37           Poultry Meat
38          

### Queste colonne richiedono un aggiusto poichè alcune righe hanno 2 prodotti cosa che non ci consente di effettuare l'incrocio dati quindi procederemo con l'addattare le colonne con il dataset delle Nazioni Unite.

### Inoltre da questi prodotti andremo ad escludere le mandrie di manzi destinati alla produzione di carne (Beef(beef herd)) e le mandrie di manzi destinati alla produzione di latticini (Beef(dairy herd)) che saranno analizzate in separate sede in quanto non sono un prodotto finito vero e proprio ma comunque hanno il loro fattore inquinante.

In [65]:
# Adattamento per favorire l'incrocio dati
cibi_temp = list()
for cibo in cibi:
     if "Beef" not in cibo:
          split = cibo.split('&')
          cibi_temp.append(str.strip(split[0]))
          if len(split) > 1:
               cibi_temp.append(str.strip(split[1]))
cibi_temp.sort()
cibi = pd.Series(cibi_temp)
cibi


0               Apples
1              Bananas
2        Barley (Beer)
3           Beet Sugar
4              Berries
5            Brassicas
6           Cane Sugar
7              Cassava
8               Cheese
9         Citrus Fruit
10              Coffee
11      Dark Chocolate
12                Eggs
13       Fish (farmed)
14              Grapes
15          Groundnuts
16                Lamb
17               Leeks
18        Maize (Meal)
19                Milk
20              Mutton
21                Nuts
22             Oatmeal
23           Olive Oil
24              Onions
25         Other Fruit
26        Other Pulses
27    Other Vegetables
28            Palm Oil
29                Peas
30            Pig Meat
31            Potatoes
32        Poultry Meat
33        Rapeseed Oil
34                Rice
35     Root Vegetables
36         Rye (Bread)
37    Shrimps (farmed)
38         Soybean Oil
39             Soymilk
40       Sunflower Oil
41                Tofu
42            Tomatoes
43         

# Incorcio dei dati
### Ovviamente l'incrocio dei dati porterà ad una diminuzione del campione il che è naturale poichè non vengono tracciati tutti i cibi che ci sono nel datset delle Nazioni Unite

### Di seguito la funzione che incorcia i dati.

In [66]:
# Funzione per incrocio dati
def incrocio_dati(df, cibi, prodotti_tracciati, colonne) -> None:
     for index, record in FAO_df[FAO_df_colonne_interessate].iterrows():
          food_type = record["Item"].lower()
          # Caso del dizionario
          if(type(cibi) == dict):
               for key in cibi.keys():
                    if food_type.startswith(cibi[key].lower()) and index not in prodotti_tracciati:
                         prodotti_tracciati.append(index)
                         FAO_df.at[index, "Type"] = key
          # Caso lista/pd.series.Series
          elif(type(cibi) == pd.core.series.Series or type(cibi) == list):
               for i in range(len(cibi)):
                    if food_type.startswith(cibi[i].lower()) and index not in prodotti_tracciati:
                         prodotti_tracciati.append(index)
                         FAO_df.at[index, "Type"] = cibi[i]
          # Caso di un tipo non permesso
          else:
               print("No type encoded")


In [67]:
prodotti_tracciati = list()
# Inserimento nel df in base al tipo di alimento(pt.1)
prodotti_tracciati = list()
incrocio_dati(FAO_df, cibi, prodotti_tracciati, colonne_interessate)
print("Numero di prodotti tracciati: ", len(prodotti_tracciati))

Numero di prodotti tracciati:  4369


# Dati non presi
### Alcuni dati non sono stati presi a causa della differente nomenclatura tra i datasets in particolare: 'Beet Sugar', 'Cane Sugar', 'Citrus Fruit', 'Fish (farmed)', 'Other Fruit', 'Other Pulses', 'Other Vegetables', 'Pig Meat', 'Sunflower Oil'.
### Effettuerò delle modifiche alle nomenclature al fine di estrapolarli.

In [68]:
colonne_dict = {
     'Beet Sugar': 'sugar beet',
     'Cane Sugar': 'sugar cane',
     'Citrus Fruit': 'citrus',
     'Fish (farmed)': 'fish',
     'Other Fruit': 'fruits, other',
     'Other Pulses': 'pulses, other and product',
     'Other Vegetables': 'vegetables, other',
     'Pig Meat': 'pigmeat',
}

incrocio_dati(FAO_df, colonne_dict, prodotti_tracciati, colonne_interessate)
print("Numero di prodotti tracciati: ", len(prodotti_tracciati))


Numero di prodotti tracciati:  6306


# Creazione del dataset 
### Ho gli indici dei prodotti che vengono tracciati e creo il dataset con i dati giusti

In [69]:
prodotti_tracciati_df = FAO_df.iloc[prodotti_tracciati]
prodotti_tracciati_df = prodotti_tracciati_df[FAO_df_colonne_interessate].reset_index()
prodotti_tracciati_df


,index,Area Abbreviation,Area,Item,Element,Unit,Y1961,Y1962,Y1963,Y1964,...,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013,Type
0,0,AFG,Afghanistan,Wheat and products,Food,1000 tonnes,1928.0,1904.0,1666.0,1950.0,...,3486.0,3704.0,4164.0,4252.0,4538.0,4605.0,4711.0,4810,4895,Wheat
1,1,AFG,Afghanistan,Rice (Milled Equivalent),Food,1000 tonnes,183.0,183.0,182.0,220.0,...,445.0,546.0,455.0,490.0,415.0,442.0,476.0,425,422,Rice
2,8,AFG,Afghanistan,Potatoes and products,Food,1000 tonnes,111.0,97.0,103.0,110.0,...,294.0,294.0,260.0,242.0,250.0,192.0,169.0,196,230,Potatoes
3,16,AFG,Afghanistan,Nuts and products,Food,1000 tonnes,2.0,3.0,1.0,1.0,...,13.0,24.0,34.0,42.0,28.0,66.0,71.0,70,44,Nuts
4,25,AFG,Afghanistan,Palm Oil,Food,1000 tonnes,0.0,0.0,0.0,0.0,...,69.0,56.0,51.0,36.0,53.0,59.0,51.0,61,64,Palm Oil
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6301,21430,ZWE,Zimbabwe,Pigmeat,Food,1000 tonnes,5.0,6.0,6.0,6.0,...,28.0,28.0,28.0,28.0,33.0,34.0,37.0,40,37,Pig Meat
6302,21437,ZWE,Zimbabwe,"Fish, Body Oil",Food,1000 tonnes,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,Fish (farmed)
6303,21438,ZWE,Zimbabwe,"Fish, Liver Oil",Food,1000 tonnes,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,Fish (farmed)
6304,21473,ZWE,Zimbabwe,"Fish, Seafood",Feed,1000 tonnes,27.0,25.0,27.0,23.0,...,4.0,9.0,6.0,9.0,5.0,15.0,15.0,15,15,Fish (farmed)


# Pulizia dei dati
### All'interno dei dati incrociati potremmo avere dei valori nulli che sporcano i dati

## Divisione per alimento

In [70]:
# Dizionario alimento => sub_df
alimenti_dict = dict()
keys = cibi.to_list()

for i in range(len(keys)):
     if keys[i] in colonne_dict.keys():
          keys[i] = colonne_dict[keys[i]].capitalize()

### Una volta fatta la divisione effettuo un filtraggio per la dividere il dataset ed effettuare una pulizia dei prodotto tracciati ed elimino quei dataset che non hanno valori

In [71]:
# Assegnazione delle chiavi a delle liste vuote
for key in keys:
     alimenti_dict[key] = list()

# Divisione dei record in base all'alimento
for index, record in prodotti_tracciati_df.iterrows():
     for key in keys:
          if record["Item"].lower().startswith(key.lower()) and index not in alimenti_dict[key]:
               alimenti_dict[key].append(index)

# Cerco le categorie vuote
prodotti_vuoti = []
for key in alimenti_dict.keys():
     if len(alimenti_dict[key]) == 0:
          prodotti_vuoti.append(key)

# Elimino le categorie di prodotto che hanno una lista di lunghezza 0
for prodotto_vuoto in prodotti_vuoti:
     alimenti_dict.pop(prodotto_vuoto, None)
alimenti_dict.keys()

for alimento in alimenti_dict.keys():
     print(alimento, " -> ", len(alimenti_dict[alimento]))
print("Tipi di alimento trovati: ", list(alimenti_dict.keys()))

Apples  ->  194
Bananas  ->  186
Sugar beet  ->  66
Sugar cane  ->  87
Cassava  ->  225
Citrus  ->  155
Coffee  ->  174
Eggs  ->  360
Fish  ->  762
Grapes  ->  173
Groundnuts  ->  170
Milk  ->  558
Mutton  ->  176
Nuts  ->  173
Olive Oil  ->  174
Onions  ->  167
Fruits, other  ->  185
Pulses, other and product  ->  255
Vegetables, other  ->  258
Palm Oil  ->  124
Peas  ->  196
Pigmeat  ->  169
Potatoes  ->  254
Poultry Meat  ->  175
Rice  ->  273
Tomatoes  ->  175
Wheat  ->  273
Wine  ->  169
Tipi di alimento trovati:  ['Apples', 'Bananas', 'Sugar beet', 'Sugar cane', 'Cassava', 'Citrus', 'Coffee', 'Eggs', 'Fish', 'Grapes', 'Groundnuts', 'Milk', 'Mutton', 'Nuts', 'Olive Oil', 'Onions', 'Fruits, other', 'Pulses, other and product', 'Vegetables, other', 'Palm Oil', 'Peas', 'Pigmeat', 'Potatoes', 'Poultry Meat', 'Rice', 'Tomatoes', 'Wheat', 'Wine']


# Effettuo la pulizia dati
### Elimino le righe cha hanno valori nulli.  

In [72]:
indici_da_eliminare = list()

for key in alimenti_dict.keys():
     for index in alimenti_dict[key]:
          if not prodotti_tracciati_df.iloc[index]["Item"].lower().startswith(key.lower()):
               indici_da_eliminare.append(int(index))

# Pulizia degli indici
for indice_da_eliminare in indici_da_eliminare:
     if indice_da_eliminare in prodotti_tracciati:
          prodotti_tracciati.remove(indice_da_eliminare)

prodotti_tracciati_df = FAO_df.iloc[prodotti_tracciati]
prodotti_tracciati_df = prodotti_tracciati_df[FAO_df_colonne_interessate].reset_index()
prodotti_tracciati_df

prodotti_tracciati_df = prodotti_tracciati_df.dropna().reset_index()
prodotti_tracciati_df.drop(columns=["level_0", "index"], inplace=True)
prodotti_tracciati_df["Prod. 1961 - 2013"] = prodotti_tracciati_df[[f"Y{i}" for i in range(1961, 2014)]].sum(axis=1)
cond = prodotti_tracciati_df["Prod. 1961 - 2013"] != 0
prodotti_tracciati_df = prodotti_tracciati_df[cond]


# Esplorazione dei dati
### Come primo step esplorativo andremo a vedere cosa viene prodotto in quantita' maggiore tra mangimi(Feed) e cibo(Food) in media dal 1961 al 2013.

In [73]:
cibi_mangimi = prodotti_tracciati_df.groupby("Element")[[f"Y{i}" for i in range(1961, 2014)]].mean()
cibi_mangimi

,Y1961,Y1962,Y1963,Y1964,Y1965,Y1966,Y1967,Y1968,Y1969,Y1970,...,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013
Element,,,,,,,,,,,,,,,,,,,,,
Feed,306.495778,312.610374,322.412545,329.761158,328.675513,342.454765,338.193004,365.063932,362.295537,352.369119,...,457.699638,454.934861,468.605549,467.661037,490.710495,501.890229,505.241255,553.435464,557.232811,568.977081
Food,296.859495,302.594157,310.489774,318.058699,327.023904,334.293227,339.982736,352.292165,358.829482,366.360425,...,884.776627,905.981939,927.877025,956.919522,979.365737,997.837450,1023.491633,1049.853386,1075.828154,1090.555113


## Anno di maggiore produzione di cibi e mangimi
### Tutti i valori vanno moltiplicati per 1000

In [74]:
magimi_max_produzione = cibi_mangimi.loc["Feed"].max()
cibo_max_produzione = cibi_mangimi.loc["Food"].max()
print(cibi_mangimi.loc["Feed"].max() * 1000, " tonnellate di mangime nell'anno", cibi_mangimi.idxmax(axis=1)[0].split("Y")[1])
print(cibi_mangimi.loc["Food"].max() * 1000, "tonnellate di cibo nell'anno", cibi_mangimi.idxmax(axis=1)[1].split("Y")[1])

568977.0808202655  tonnellate di mangime nell'anno 2013
1090555.112881806 tonnellate di cibo nell'anno 2013


### I valori più elevati corrispondono con il 2013 che è anche l'ultimo anno a disposizione.
# Incremento produzione cibi e mangimi
### Vedremo l'aumento della produzione di cibi anno per anno, ogni colonna ci dice di quanto è aumentata la produzione rispetto all'anno precedente.
# Cibi

In [75]:
Food = cibi_mangimi.loc["Food"]

Food_yearly_increment = [abs(Food[f"Y{i}"] - Food[f"Y{i+1}"]) for i in range(1961, 2014) if i + 1 < 2014]
Food_yearly_increment_df = pd.DataFrame()
for i in range(1961, 2014):
     if i+1 < 2014:
          Food_yearly_increment_df[f"Y{i}/Y{i+1}"] = pd.Series(Food_yearly_increment[abs(1961 - i)])

print(f"Incremento annuo medio della produzione di cibo = {Food_yearly_increment_df.iloc[0].mean() * 1000} tonnellate")

Incremento annuo medio della produzione di cibo = 15263.377260190007 tonnellate


# Mangimi

In [76]:
Feed = cibi_mangimi.loc["Feed"]

Feed_yearly_increment = [abs(Feed[f"Y{i}"] - Feed[f"Y{i+1}"]) for i in range(1961, 2014) if i + 1 < 2014]
Feed_yearly_increment_df = pd.DataFrame()
for i in range(1961, 2014):
     if i+1 < 2014:
          Feed_yearly_increment_df[f"Y{i}/Y{i+1}"] = pd.Series(Feed_yearly_increment[abs(1961 - i)])

print(f"Incremento annuo medio della produzione di mangimi = {Feed_yearly_increment_df.iloc[0].mean() * 1000} tonnellate")

Incremento annuo medio della produzione di mangimi = 11787.02329034054 tonnellate


# Cibi piu' e meno prodotti dal 1961 al 2013
### Scopriremo quali sono i 5 cibi piu' prodotti ed i 5 cibi meno prodotti dal 1961 al 2013.

In [77]:
columns_interested = ["Item", "Element", "Area", "Prod. 1961 - 2013", "Type"]
Food_items = prodotti_tracciati_df[columns_interested]
Food_items = Food_items[Food_items["Element"] == "Food"]
Food_items = Food_items.sort_values(by=["Prod. 1961 - 2013"], ascending=False).reset_index().drop(columns=["index"]).drop_duplicates().reset_index().drop(columns=["index"])
Food_items["Tonnes"] = Food_items["Prod. 1961 - 2013"] * 1000
Food_items

,Item,Element,Area,Prod. 1961 - 2013,Type,Tonnes
0,"Vegetables, Other",Food,"China, mainland",7962498.0,Other Vegetables,7.962498e+09
1,Rice (Milled Equivalent),Food,"China, mainland",4271067.0,Rice,4.271067e+09
2,Wheat and products,Food,"China, mainland",3521951.0,Wheat,3.521951e+09
3,Milk - Excluding Butter,Food,United States of America,3353075.0,Milk,3.353075e+09
4,Rice (Milled Equivalent),Food,India,3096326.0,Rice,3.096326e+09
...,...,...,...,...,...,...
3471,Peas,Food,Nigeria,1.0,Peas,1.000000e+03
3472,"Citrus, Other",Food,Madagascar,1.0,Citrus Fruit,1.000000e+03
3473,"Citrus, Other",Food,Congo,1.0,Citrus Fruit,1.000000e+03
3474,Grapes and products (excl wine),Food,Grenada,1.0,Grapes,1.000000e+03


# Mangimi più e meno prodotti dal 1961 al 2013
### Scopriremo quali sono i 5 mangimi più prodotti ed i 5 mangimi meno prodotti dal 1961 al 2013.

In [78]:
Feed_items = prodotti_tracciati_df[columns_interested]
Feed_items = Feed_items[Feed_items["Element"] == "Feed"]
Feed_items = Feed_items.sort_values(by=["Prod. 1961 - 2013"], ascending=False).reset_index().drop(columns=["index"]).drop_duplicates().reset_index().drop(columns=["index"])
Feed_items["Tonnes"] = Feed_items["Prod. 1961 - 2013"] * 1000
Feed_items

,Item,Element,Area,Prod. 1961 - 2013,Type,Tonnes
0,Potatoes and products,Feed,Poland,830938.0,Potatoes,830938000.0
1,Cassava and products,Feed,Brazil,644635.0,Cassava,644635000.0
2,Rice (Milled Equivalent),Feed,"China, mainland",491505.0,Rice,491505000.0
3,Cassava and products,Feed,Nigeria,483039.0,Cassava,483039000.0
4,Potatoes and products,Feed,"China, mainland",443362.0,Potatoes,443362000.0
...,...,...,...,...,...,...
745,Rice (Milled Equivalent),Feed,Vanuatu,1.0,Rice,1000.0
746,"Fish, Seafood",Feed,"China, Macao SAR",1.0,Fish (farmed),1000.0
747,Milk - Excluding Butter,Feed,Antigua and Barbuda,1.0,Milk,1000.0
748,"Fish, Seafood",Feed,Uganda,1.0,Fish (farmed),1000.0


# Inquinamento da CO2 dei cibi più prodotti

In [79]:
# Prendo le colonne relative all'inquinamento da CO2
cibo_max_produzione = Food_items.iloc[:5]
cibo_max_type = Food_items.iloc[:5]["Type"].to_list()
cibo_max_type
# Passaggio da tonnellate in KG bisogna moltimplicare per 1000
Food_CO2_total_emissions = [print(f"{record['Food product']}: {record['Total_emissions']}") for index, record in Food_Production_df.iterrows() if record["Food product"] in cibo_max_type]
# cibo_max_produzione.loc["KGCO2"] = cibo_max_produzione["Tonnes"] * 1000
# Food_CO2_total_emissions
# Non prende Wheat perchè all'interno del df Wheat è compreso nella voce Wheat and Rye quindi bisogna andare a rislvere la nomenclatura(Controlla sopra se non lo hai già fatto).
# Fare un po di refactoring e tirare  fuori almeno una funzione per la pulizia dei dati.

Rice: 4.0
Other Vegetables: 0.5
Milk: 2.8000000000000003
